Useful links:
 - https://stackoverflow.com/questions/19846332/python-threading-inside-a-class/19846691
 - https://stackoverflow.com/questions/39145357/python-error-socket-error-errno-11-resource-temporarily-unavailable-when-s 

In [1]:
# Research while working on 1020 project
import socket
import threading
import time

In [2]:
# Constants
HEADERSIZE = 10

In [3]:
class server:
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.socket.bind((self.ip, self.port))
        self.socket.listen(5)
        
        self.accepting_thread = None
        self.client_sockets = []
        self.client_threads = []
        
        self.stop_threads = False
    
    def broadcast(self, message):
        packet = wrap_header(message)
        for socket in self.client_sockets:
            print(packet)
            socket.send(packet)
            #socket.send(packet)
    
    def _accept_clients(self):
        while not self.stop_threads:
            sockfd, addr = self.socket.accept()
            self.client_sockets.append(sockfd)
            #self.client_threads.append(threading.Thread(target=self._client_thread, args=(sockfd,), daemon=True))
            #self.client_threads[-1].start()
            print(self.client_sockets)
        
    def accept_clients(self):
        self.accepting_thread = threading.Thread(target=self._accept_clients, daemon=True)
        self.accepting_thread.start()
        
    def _client_thread(self, socketfd):
        while not self.stop_threads:
            data = bytes("", 'utf-8')
            try:
                print(socketfd.recv(HEADERSIZE))
                incoming_size = int(socketfd.recv(HEADERSIZE))
                print(incoming_size)
                data = socketfd.recv(incoming_size)
                #while len(data) < incoming_size:
                #    data += socketfd.recv(16)
                #if len(data) == incoming_size:
                #    print(str(data))
            except Exception as e:
                print("GOLDEN GOOSE")
                print(e)
                #socketfd.close()
                continue
    
    def _message_handler(self):
        while True:
            to_read, to_write, errors = select.select(self.client_sockets, [], [])
            print("To read is ready: {}".format(to_read))
            for s in to_read:
                if s == self.socket:
                    print("Well something's gone wrong")
                else:
                    try:
                        print(s.recv(1024))
                    except Exception as e:
                        print("message handler hit bump")
                        print(e)
                        continue
    def handle_messages(self):
        self.message_handler_thread = threading.Thread(target=self._message_handler, daemon=True)
    
    def stop(self):
        self.stop_threads = True


class client:
    def __init__(self, username, ip, port):
        self.username = username
        self.server_address = (ip, port)
        self.ip = ip
        self.port = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect(self.server_address)
        self.socket.setblocking(False)
        
        self.messages_thread = threading.Thread(target=self._messages_thread, daemon=True)
        self.messages_thread.start()
    
    def send_message(self, message):
        """Method to send a packet. Takes string as argument"""
        print("This ran. Send message.")
        #self.socket.send(create_packet(message))
        self.socket.send(message)
    
    def _messages_thread(self):
        print("Messages thread was started")
        while True:
            try:
                incoming_size = int(self.socket.recv(HEADERSIZE))
                #data = socketfd.recv(incoming_size)
                while len(data) < incoming_size:
                    data += self.socket.recv(16)
                if len(data) == incoming_size:
                    print(str(data))
                    return data
            except Exception as e:
                self.socket.close()
                print(e)
                break

In [4]:
# misc functions
#def create_packet(obj):
##    compressed_text = pickle.dumps(obj)
    #compressed_text = bytes(text, 'utf-8')
#    length = len(compressed_text)
#    return bytes(f"{length:<{HEADERSIZE}}", 'utf-8') + compressed_text

def wrap_header(text):
    data = bytes(text, 'utf-8')
    length = len(data)
    return bytes(f"{length:<{HEADERSIZE}}", 'utf-8') + data

def receive_packet(socketfd):
    data = bytes("", 'utf-8')
    packet_length = int(socketfd.recv(HEADERSIZE))
    print("WE GOT HERE")
    data = socketfd.recv(packet_length)
    #while len(data) < packet_length:
    #    data += socketfd.recv(packet_length)
    if len(data) == packet_length:
        return pickle.loads(data)
    else:
        return None

In [5]:
# Testing
test_server = server('127.0.0.1', 1242)
test_server.accept_clients()
test_server.handle_messages()
time.sleep(1)





In [6]:
test_client = client("user", '127.0.0.1', 1242)

[<socket.socket fd=42, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1242), raddr=('127.0.0.1', 48962)>]
Messages thread was started
[Errno 11] Resource temporarily unavailable


In [7]:
test_server.broadcast('Hello')

b'5         Hello'


In [8]:
test_server.broadcast("Hello")

b'5         Hello'


BrokenPipeError: [Errno 32] Broken pipe

[<socket.socket fd=42, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1242)>, <socket.socket fd=46, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1242), raddr=('127.0.0.1', 48966)>]
